In [1]:
import os
import json
import re
from langchain_neo4j import Neo4jGraph, GraphCypherQAChain
from langchain_ollama import ChatOllama
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_core.documents import Document
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

In [2]:
os.environ["NEO4J_URI"] = "bolt://localhost:7687"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "neo4j_password"

In [10]:
# Set LLM
llm = ChatOllama(model="llama3.2", temperature=0)

In [9]:
# Load data
with open("data/diabetes_subset.json", "r") as f:
    json_data = json.load(f)

questions = [entry.get("QUESTION", "N/A") for entry in json_data.values()]
contexts = [entry.get("CONTEXTS", "N/A") for entry in json_data.values()]
conclusions = [entry.get("LONG_ANSWER", "N/A") for entry in json_data.values()]
ground_truth = [entry.get("final_decision", "N/A") for entry in json_data.values()]


### Extract as many relations as possible ###

In [ ]:
entity_prompt = PromptTemplate(
    input_variables=["question"],
    template="""
        Attempt to extract as many entities as you can. Maintain
        Entity Consistency: When extracting entities, it's vital to ensure
        consistency. If an entity, such as "John Doe", is mentioned multiple
        times in the text but is referred to by different names or pronouns
        (e.g., "Joe", "he"), always use the most complete identifier for
        that entity. The knowledge graph should be coherent and easily "
        understandable, so maintaining consistency in entity references is "
        crucial.
        IMPORTANT NOTES:\n- Don't add any explanation and text.:
Question: "{question}"
Entities (comma-separated):"""
)

entity_chain = entity_prompt | llm

In [ ]:
relation_prompt = PromptTemplate(
    input_variables = ["abstract"],
    template="""
        Attempt to extract as many relations as you can. 
        IMPORTANT NOTES:\n- Don't add any explanation and text.:
Question: "{abstract}"
Relations (comma-separated):"""
)

ValidationError: 2 validation errors for PromptTemplate
input_variables
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing
template
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing